In [ ]:
from src.material.textures.noise.perlin_noise import PerlinNoise
from src.render.loops.progress import PreviewConfig, ProgressDisplay
from src.render.render_config import RenderConfig
from src.material.material.phong_material import PhongMaterial
from src.material.color import Color
from src.geometry.primitives.sphere import Sphere
from src.scene.primitive import Object
from src.scene.scene import Scene
from src.scene.camera import Camera
from src.scene.light import PointLight
from src.math import Vertex, Vector
from src.render.resolution import Resolution


In [ ]:
glossy_red = PhongMaterial(
    name="glossy_red",
    base_color=Color.custom_rgb(255, 10, 40),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=30,
    normal_noise=PerlinNoise(
        scale=1.5,
        strength=0.2
    ),
)

scene = Scene(
    camera=Camera(
        fov=50,
        aspect_ratio=Resolution.R360p.aspect_ratio,
        origin=Vertex(0, 0, 0),
        direction=Vector(0, 0, -1)
    ),

    primitives = [
        Object(
            geometry=Sphere(
                center=Vertex(0.0, 0.0, -6),
                radius=2.0
            ),
            material=glossy_red,
        ),
    ],

    lights=[
        PointLight(
            position=Vertex(5, 5, 0),
            intensity=1000.0,
            falloff=0.01
        ),
    ],
)

preview_configuration = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

render_configuration = RenderConfig(
    resolution=Resolution.R360p,
    samples_per_pixel=1,
    max_depth=2,
)

In [ ]:
from src import NormalShader
from src.render import LinearRayCaster

shader = NormalShader()

my_ray_tracer = LinearRayCaster(scene=scene, render_config=render_configuration, preview_config=preview_configuration, shading_model=shader)
png_path = my_ray_tracer.render("images/test.png")

In [ ]:
from src.material.textures.noise.FBM import FBMNoise

normal_noise = FBMNoise(scale=2.0, strength=0.2, octaves=6, lacunarity=2.0, gain=0.5)

glossy_red_fbm = PhongMaterial(
    name="glossy_red_fbm",
    base_color=Color.custom_rgb(255, 10, 40),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=30,
    normal_noise=normal_noise,
)

scene_fbm = Scene(
    camera=Camera(
        fov=50,
        aspect_ratio=Resolution.R360p.aspect_ratio,
        origin=Vertex(0, 0, 0),
        direction=Vector(0, 0, -1)
    ),

    primitives = [
        Object(
            geometry=Sphere(
                center=Vertex(0.0, 0.0, -6),
                radius=2.0
            ),
            material=glossy_red_fbm,
        ),
    ],

    lights=[
        PointLight(
            position=Vertex(5, 5, 0),
            intensity=1000.0,
            falloff=0.01
        ),
    ],
)

my_ray_tracer_fbm = LinearRayCaster(scene=scene_fbm, render_config=render_configuration, preview_config=preview_configuration, shading_model=shader)
png_path_fbm = my_ray_tracer_fbm.render("images/test_fbm.png")


In [ ]:
from src.material.textures.noise.turbulence_noise import TurbulenceNoise

normal_noise_turbulence = TurbulenceNoise(scale=2.0, strength=0.5, octaves=16, lacunarity=2, gain=0.2)

glossy_red_turbulence = PhongMaterial(
    name="glossy_red_turbulence",
    base_color=Color.custom_rgb(255, 10, 40),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=30,
    normal_noise=normal_noise_turbulence,
)

scene_turbulence = Scene(
    camera=Camera(
        fov=50,
        aspect_ratio=Resolution.R360p.aspect_ratio,
        origin=Vertex(0, 0, 0),
        direction=Vector(0, 0, -1)
    ),

    primitives = [
        Object(
            geometry=Sphere(
                center=Vertex(0.0, 0.0, -6),
                radius=2.0
            ),
            material=glossy_red_turbulence,
        ),
    ],

    lights=[
        PointLight(
            position=Vertex(5, 5, 0),
            intensity=1000.0,
            falloff=0.01
        ),
    ],
)

my_ray_tracer_turbulence = LinearRayCaster(scene=scene_turbulence, render_config=render_configuration, preview_config=preview_configuration, shading_model=shader)
png_path_turbulence = my_ray_tracer_turbulence.render("images/test_turbulence.png")

In [ ]:
from src.material.textures.noise.normal_base import Noise
from dataclasses import dataclass, field


@dataclass
class BigRockNoise(Noise):
    base: PerlinNoise = field(default_factory=PerlinNoise)
    octaves: int = 5
    lacunarity: float = 2.0
    gain: float = 0.5
    ridge_sharpness: float = 1.8  # >1 makes sharper ridges

    def value(self, p: Vertex | Vector) -> float:
        x = (p + self.offset) * self.scale

        amp, freq = 1.0, 1.0
        total, amp_sum = 0.0, 0.0

        for _ in range(self.octaves):
            n = self.base.value(x * freq)      # ~[-1,1]
            n = 1.0 - abs(n)                   # ridges [0,1]
            n = n ** self.ridge_sharpness      # sharpen
            total += amp * n
            amp_sum += amp
            amp *= self.gain
            freq *= self.lacunarity

        total = total / amp_sum if amp_sum > 0 else total
        return total * self.strength

normal_noise = BigRockNoise(scale=0.7, strength=0.25, octaves=5)

big = BigRockNoise(scale=0.7, strength=0.22, octaves=5)
small = TurbulenceNoise(scale=6.0, strength=0.06, octaves=3)

# easiest: wrap into one noise
@dataclass
class MixNoise(Noise):
    a: Noise = field(default_factory=Noise)   # replace in init
    b: Noise = field(default_factory=Noise)
    mix: float = 0.8  # 0..1 more = mostly a

    def value(self, p):
        return self.mix * self.a.value(p) + (1.0 - self.mix) * self.b.value(p)


@dataclass
class DomainWarpNoise(Noise):
    base: Noise = field(default_factory=lambda: BigRockNoise())
    warp: PerlinNoise = field(default_factory=PerlinNoise)
    warp_scale: float = 0.6     # frequency of warp
    warp_amount: float = 0.8    # how much to distort

    def value(self, p: Vertex | Vector) -> float:
        x = (p + self.offset) * self.scale

        wx = self.warp.value(x * self.warp_scale + Vector(31.7, 12.3, 5.1))
        wy = self.warp.value(x * self.warp_scale + Vector(7.4,  19.9, 2.8))
        wz = self.warp.value(x * self.warp_scale + Vector(13.2, 3.3,  27.1))

        warped = x + Vector(wx, wy, wz) * self.warp_amount
        return self.base.value(warped) * self.strength


normal_noise = DomainWarpNoise(
    base=BigRockNoise(scale=0.8, strength=3.0, octaves=5),
    warp_scale=0.6,
    warp_amount=0.9,
    strength=0.22,
    scale=1.0
)


glossy_red.normal_noise = BigRockNoise(scale=0.8, strength=0.25, octaves=5, eps=1e-2)


scene.primitives[0].material = glossy_red
my_ray_tracer_rock = LinearRayCaster(scene=scene, render_config=render_configuration, preview_config=preview_configuration, shading_model=shader)
png_path_rock = my_ray_tracer_rock.render("images/test_rock.png")


In [ ]:
# planet_scene.py
# Nice-looking procedural planet: oceans + land + mountains + ice caps
# Uses your existing engine classes (Scene, Camera, Sphere, Primitive, PointLight/DirectionalLight/AmbientLight,
# RenderConfig/PreviewConfig/LinearRayCaster, Color/Vector/Vertex, BlinnPhong-style shading).
from dataclasses import dataclass, field

# --- your project imports (adjust paths if needed) ---
from src.scene.scene import Scene
from src.scene.camera import Camera
from src.scene.primitive import Object
from src.scene.light import DirectionalLight, AmbientLight, Light, LightType
from src.scene.surface_interaction import SurfaceInteraction
from src.material.material.phong_material import PhongMaterial
from src.material.color import Color
from src.material.textures.noise.normal_base import Noise
from src.math import Vector, Vertex
from src.shading.helpers import shadow_trace, light_dir_dist
from src.shading.helpers import tangent_basis
from src.math import fresnel_schlick, dielectric_f0


# ============================================================
# NOISE STACK: continents + mountains + domain warp
# ============================================================

@dataclass
class ContinentalMask(Noise):
    base: PerlinNoise = field(default_factory=PerlinNoise)
    octaves: int = 3
    lacunarity: float = 2.0
    gain: float = 0.5
    sea_level: float = 0.50

    def value(self, p: Vertex | Vector) -> float:
        x = (p + self.offset) * self.scale

        amp, freq = 1.0, 1.0
        total, amp_sum = 0.0, 0.0

        for _ in range(self.octaves):
            total += amp * self.base.value(x * freq)
            amp_sum += amp
            amp *= self.gain
            freq *= self.lacunarity

        total = total / amp_sum if amp_sum > 0 else total  # ~[-1,1]
        h = 0.5 * total + 0.5                              # [0,1]

        if h <= self.sea_level:
            return 0.0
        return (h - self.sea_level) / (1.0 - self.sea_level)  # [0,1] on land


@dataclass
class PerlinMountainNoise(Noise):
    base: PerlinNoise = field(default_factory=PerlinNoise)
    octaves: int = 5
    lacunarity: float = 2.0
    gain: float = 0.5
    sharpness: float = 1.6

    def value(self, p: Vertex | Vector) -> float:
        x = (p + self.offset) * self.scale

        amp, freq = 1.0, 1.0
        total, amp_sum = 0.0, 0.0

        for _ in range(self.octaves):
            n = self.base.value(x * freq)   # ~[-1,1]
            n = 1.0 - abs(n)                # ridge-like
            n = n ** self.sharpness         # sharpen peaks
            total += amp * n
            amp_sum += amp
            amp *= self.gain
            freq *= self.lacunarity

        return total / amp_sum if amp_sum > 0 else total   # ~[0,1]


@dataclass
class TerrainHeight(Noise):
    continent: Noise = field(default_factory=ContinentalMask)
    mountains: Noise = field(default_factory=PerlinMountainNoise)

    def value(self, p: Vertex | Vector) -> float:
        x = (p + self.offset) * self.scale
        c = self.continent.value(x)   # 0..1
        m = self.mountains.value(x)   # 0..1

        # "land height" (0..1) with mountains shaping
        mountain_shape = m ** 1.3
        h = c * (0.15 + 1.20 * mountain_shape)
        return h * self.strength


@dataclass
class DomainWarpNoise(Noise):
    base: Noise = field(default_factory=TerrainHeight)
    warp: PerlinNoise = field(default_factory=PerlinNoise)
    warp_scale: float = 1.2
    warp_amount: float = 0.25

    def value(self, p: Vertex | Vector) -> float:
        x = (p + self.offset) * self.scale

        wx = self.warp.value(x * self.warp_scale + Vector(31.7, 12.3, 5.1))
        wy = self.warp.value(x * self.warp_scale + Vector(7.4, 19.9, 2.8))
        wz = self.warp.value(x * self.warp_scale + Vector(13.2, 3.3, 27.1))

        warped = x + Vector(wx, wy, wz) * self.warp_amount
        return self.base.value(warped) * self.strength


# ============================================================
# PLANET: albedo + normal bump based on spherical mapping
# ============================================================

def clamp01(x: float) -> float:
    return 0.0 if x < 0.0 else (1.0 if x > 1.0 else x)

def lerp(a: Color, b: Color, t: float) -> Color:
    t = clamp01(t)
    return a * (1.0 - t) + b * t


@dataclass
class PlanetModel:
    # height field for land (0..1)
    terrain: Noise
    # separate mountain detail for bump
    mountains: Noise
    # separate continent mask (0..1 land)
    continents: Noise

    # colors (pick anything)
    ocean_deep: Color = field(default_factory=lambda: Color.custom_rgb(10, 35, 90))
    ocean_shallow: Color = field(default_factory=lambda: Color.custom_rgb(20, 70, 140))
    land_low: Color = field(default_factory=lambda: Color.custom_rgb(30, 120, 60))
    land_high: Color = field(default_factory=lambda: Color.custom_rgb(120, 140, 80))
    rock: Color = field(default_factory=lambda: Color.custom_rgb(120, 120, 120))
    snow: Color = field(default_factory=lambda: Color.custom_rgb(235, 235, 245))

    def sample(self, p_unit: Vector) -> tuple[Color, float, float]:
        """
        p_unit: unit sphere coordinate (use hit.normal.normalize()).
        returns: (albedo_color, land_mask_0to1, height_0to1)
        """
        # lat for ice caps
        lat = abs(p_unit.y)  # 0 equator -> 1 poles

        land = clamp01(self.continents.value(p_unit))     # 0..1
        h = clamp01(self.terrain.value(p_unit))           # 0..1 on land, ~0 ocean

        # ocean gradient by "negative height" proxy (since ocean has h ~ 0)
        ocean = lerp(self.ocean_deep, self.ocean_shallow, 1.0 - clamp01(lat * 0.8))

        # land gradient by height
        plains = lerp(self.land_low, self.land_high, h)
        mountains = lerp(self.rock, self.snow, clamp01((h - 0.65) / 0.35))

        # choose land vs ocean
        land_color = lerp(plains, mountains, clamp01((h - 0.40) / 0.25))


        # snow caps by latitude (stronger near poles, but only on land)
        ice = clamp01((lat - 0.72) / 0.20) * land
        land_color = lerp(land_color, self.snow, ice)

        albedo = lerp(ocean, land_color, land)

        return albedo, land, h


# ============================================================
# NORMAL PERTURBATION: sphere-mapped bumps
# ============================================================

def apply_planet_bump(hit: SurfaceInteraction, material: PhongMaterial, n: Vector, planet: PlanetModel) -> Vector:
    """
    Bump normals using mountains only on land.
    Uses unit-sphere mapping: p_unit = hit.normal.normalize().
    """
    bump_strength = getattr(material, "bump_strength", 0.12)  # add this attr to material, or keep default
    if bump_strength <= 0.0:
        return n

    # sphere mapping coordinate (no geometry center needed)
    p_unit = hit.normal.normalize()

    land = clamp01(planet.continents.value(p_unit))
    if land <= 0.0:
        return n  # no bumps in ocean

    # higher freq detail for bumps
    scale = getattr(material, "bump_scale", 10.0)
    eps = getattr(material, "bump_eps", 1e-3)
    inv_eps = 1.0 / eps

    tangent, bitangent = tangent_basis(n)

    # sample "mountain" heightfield
    h0 = planet.mountains.value(p_unit * scale)
    ht = planet.mountains.value((p_unit + tangent * eps) * scale)
    hb = planet.mountains.value((p_unit + bitangent * eps) * scale)

    dht = (ht - h0) * inv_eps
    dhb = (hb - h0) * inv_eps

    # land-masked bump
    s = bump_strength * land
    return (n - tangent * (s * dht) - bitangent * (s * dhb)).normalize()


# ============================================================
# SHADER: Blinn-Phong but base_color comes from planet sampling
# ============================================================

class PlanetBlinnPhongShader:
    def __init__(self, planet: PlanetModel, use_fresnel: bool = False) -> None:
        self.planet = planet
        self.use_fresnel = use_fresnel

    def shade(self, hit: SurfaceInteraction, light: Light, view_dir: Vector, scene: Scene | None = None) -> Color:
        if scene is None:
            raise ValueError("Scene must be provided for shading.")

        material: PhongMaterial = hit.material

        # sphere mapping coordinate
        p_unit = hit.normal.normalize()

        # planet albedo
        albedo, land, h = self.planet.sample(p_unit)

        # ambient
        if light.type == LightType.AMBIENT:
            return albedo * light.intensity_at(hit.point)

        # shadow + light direction
        light_direction, light_distance = light_dir_dist(hit, light)
        if shadow_trace(hit, light_direction, light_distance, scene=scene):
            return Color(0.0, 0.0, 0.0)

        light_intensity = light.intensity_at(hit.point)
        if light_intensity <= 0.0:
            return Color(0.0, 0.0, 0.0)

        n = hit.normal.normalize()
        n = apply_planet_bump(hit, material, n, self.planet)

        l = light_direction.normalize()
        v = view_dir.normalize()

        diffuse = self._lambert(albedo, n, l)
        specular = self._blinn_spec(material, n, l, v, land)

        return (diffuse + specular) * light_intensity

    def shade_multiple_lights(self, hit: SurfaceInteraction, lights: list[Light], view_dir: Vector, scene: Scene | None = None) -> Color:
        acc = Color(0, 0, 0)
        for L in lights:
            acc += self.shade(hit, L, view_dir, scene=scene)
        return acc

    @staticmethod
    def _lambert(albedo: Color, n: Vector, l: Vector) -> Color:
        ndotl = max(0.0, n.dot(l))
        return albedo * ndotl

    def _blinn_spec(self, material: PhongMaterial, n: Vector, l: Vector, v: Vector, land: float) -> Color:
        # make ocean a bit shinier than land
        ocean_shine_boost = 1.6 * (1.0 - land)
        shininess = max(2.0, float(getattr(material, "shininess", 64.0)) * ocean_shine_boost)

        h = (l + v).normalize()
        ndoth = max(0.0, n.dot(h))

        spec_color = getattr(material, "spec_color", Color.custom_rgb(255, 255, 255))
        spec = spec_color * (ndoth ** shininess)

        if self.use_fresnel and getattr(material, "ior", 1.0) > 1.0:
            dielectric_color = dielectric_f0(material.get_ior())
            F = fresnel_schlick(n, v, dielectric_color)
            spec = spec * F

        return spec


# ============================================================
# BUILD SCENE
# ============================================================

# --- planet noises (sphere-mapped via hit.normal.normalize()) ---
# continents
continents = ContinentalMask(scale=1.05, sea_level=0.50)

# big mountain chains
mountains = PerlinMountainNoise(
    scale=14.0,
    octaves=6,
    sharpness=1.9
)

terrain = TerrainHeight(
    continent=continents,
    mountains=mountains,
    strength=1.0
)

terrain_warped = DomainWarpNoise(
    base=terrain,
    warp_scale=1.5,
    warp_amount=0.45,
    strength=1.0
)

planet = PlanetModel(
    terrain=terrain_warped,
    mountains=PerlinMountainNoise(scale=20.0, octaves=5, sharpness=2.0),
    continents=continents
)

shader = PlanetBlinnPhongShader(planet=planet, use_fresnel=False)

# --- material (used mainly for spec + bump params) ---
planet_material = PhongMaterial(
    name="planet",
    base_color=Color.custom_rgb(255, 255, 255),   # ignored by planet shader
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=64,
)
# bump tuning (read by apply_planet_bump)
setattr(planet_material, "bump_strength", 0.28)
setattr(planet_material, "bump_scale", 20.0)
setattr(planet_material, "bump_eps", 1e-3)

# --- geometry ---
planet_sphere = Sphere(center=Vertex(0.0, 0.0, -6.0), radius=2.0)

scene = Scene(
    camera=Camera(
        fov=45,
        aspect_ratio=1.0,  # square looks nice for planet
        origin=Vertex(0, 0, 0),
        direction=Vector(0, 0, -1)
    ),
    primitives=[
        Object(
            geometry=planet_sphere,
            material=planet_material,
        ),
    ],
    lights=[
        # soft fill so night side isn't pure black
        AmbientLight(intensity=0.08),

        # "sun"
        DirectionalLight(
            intensity=2.2,
            direction=Vector(-1.0, -0.4, -0.8)
        ),
    ],
)

preview_configuration = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

render_configuration = RenderConfig(
    resolution=Resolution.R360p,
    samples_per_pixel=1,
    max_depth=2,
)

# ============================================================
# RENDER
# ============================================================

raycaster = LinearRayCaster(
    scene=scene,
    render_config=render_configuration,
    preview_config=preview_configuration,
    shading_model=shader
)

png_path = raycaster.render("images/planet.png")
print("Saved:", png_path)
